In [1]:
!pip install sentence-transformers
import pandas as pd
import time
from tqdm import tqdm
import seaborn as sns
import numpy as np
from textblob import TextBlob
import matplotlib.pyplot as plt
from sentence_transformers import SentenceTransformer,CrossEncoder

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 85 kB 4.7 MB/s 
     |████████████████████████████████| 5.5 MB 63.9 MB/s 
     |████████████████████████████████| 1.3 MB 57.5 MB/s 
     |████████████████████████████████| 163 kB 72.1 MB/s 
     |████████████████████████████████| 7.6 MB 53.7 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=a6dd7d3870a9fcd57c9395e5c80196faf9e395c04f1ef424ae21eb7a1a7dcf8d
  Stored in directory: /root/.cache/pip/wheels/bf/06/fb/d59c1e5bd1dac7f6cf61ec0036cc3a10ab8fecaa6b2c3d3ee9
Successfully built sentence-transformers


In [2]:
import os
for dirname, _, filenames in os.walk('input/'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

input/concepts.csv
input/object_property.csv
input/process_name.csv
input/process_name_data_property.csv


In [3]:
data_concepts = pd.read_csv('input/concepts.csv')
data_process_name = pd.read_csv('input/process_name.csv')
data_process_name_data_property = pd.read_csv('input/process_name_data_property.csv')
data_object_property = pd.read_csv('input/object_property.csv')


In [4]:
list_process_name=[]
for elm in data_process_name['process_name']:
  list_process_name.append(elm)
list_process_name

[' plan scope management',
 ' collect requirements',
 ' define scope',
 ' create wbs',
 ' validate scope',
 ' control scope',
 ' plan schedule management',
 ' define activities',
 ' sequence activities',
 ' estimate activity durations',
 ' develop schedule',
 ' control schedule',
 ' plan cost management',
 ' estimate costs',
 ' determine budget',
 ' control costs']

In [5]:
list_process_name_figure=[]
for elm in list_process_name:
  text=''
  for i in data_process_name_data_property.index:
    if data_process_name_data_property.loc[i,'Domaine']==elm:
      text+=data_process_name_data_property.loc[i,'Range']+' , '
  list_process_name_figure.append(text[:-2])
list_process_name_figure

['Figure 5-3 , Figure 5-2 ',
 'Figure 5-5 , Figure 5-4 ',
 'Figure 5-8 , Figure 5-9 ',
 'Figure 5-10 , Figure 5-11 ',
 'Figure 5-16 , Figure 5-15 ',
 'Figure 5-17 , Figure 5-18 ',
 'Figure 6-3 , Figure 6-4 ',
 'Figure 6-5 , Figure 6-6 ',
 'Figure 6-8 , Figure 6-7 ',
 'Figure 6-13 , Figure 6-12 ',
 'Figure 6-15 , Figure 6-14 ',
 'Figure 6-22 , Figure 6-23 ',
 'Figure 7-3 , Figure 7-2 ',
 'Figure 7-4 , Figure 7-5 ',
 'Figure 7-6 , Figure 7-7 ',
 'Figure 7-10 , Figure 7-11 ']

In [6]:
list_process_name_inputs=[]
list_process_name_outputs=[]
list_process_name_technics_and_tools=[]

for elm in list_process_name:
  text_in=''
  text_ou=''
  text_tt=''
  for i in data_concepts.index:
    if data_concepts.loc[i,'process_name']==elm and data_concepts.loc[i,'type']=='Has_Inputs':
      text_in+=data_concepts.loc[i,'concepts']+' , '
    if data_concepts.loc[i,'process_name']==elm and data_concepts.loc[i,'type']=='Has_Outputs':
      text_ou+=data_concepts.loc[i,'concepts']+' , '
    if data_concepts.loc[i,'process_name']==elm and data_concepts.loc[i,'type']=='Has_Technics_Tools':
      text_tt+=data_concepts.loc[i,'concepts']+' , '
  list_process_name_inputs.append(text_in[:-2])
  list_process_name_outputs.append(text_ou[:-2])
  list_process_name_technics_and_tools.append(text_tt[:-2])


print('inputs: ',list_process_name_inputs)
print('outputs: ',list_process_name_outputs)
print('technics & tools: ',list_process_name_technics_and_tools)

inputs:  [' project charter ,  project management plan ,  enterprise environmental factors ,  organizational process assets ', ' project charter ,  project management plan ,  project documents ,  business documents ,  agreements ,  enterprise environmental factors ,  organizational process assets ', ' project charter ,  project management plan ,  project documents ,  enterprise environmental factors ,  organizational process assets ', ' project management plan ,  project documents ,  enterprise environmental factors ,  organizational process assets ', '', ' project management plan ,  project documents ,  work performance data ,  organizational process assets ', ' project charter ,  project management plan ,  enterprise environmental factors ,  organizational process assets ', ' project management plan ,  enterprise environmental factors ,  organizational process assets ', ' project management plan ,  project documents ,  enterprise environmental factors ,  organizational process assets

In [7]:
data_process_name['Data_property']=list_process_name_figure
data_process_name['inputs']=list_process_name_inputs
data_process_name['outputs']=list_process_name_outputs
data_process_name['technics_and_tools']=list_process_name_technics_and_tools
data_process_name

,process_name,Definition,synonyms,Data_property,inputs,outputs,technics_and_tools
0,plan scope management,plan scope management is the process of creati...,program range direction,"Figure 5-3 , Figure 5-2","project charter , project management plan , ...","scope management plan , requirements managem...","expert judgment , data analysis , meetings"
1,collect requirements,collect requirements is the process of determi...,hoard requirement,"Figure 5-5 , Figure 5-4","project charter , project management plan , ...","requirements documentation , requirements tr...","expert judgment , data gathering , data ana..."
2,define scope,define scope is the process of developing a de...,delimit range,"Figure 5-8 , Figure 5-9","project charter , project management plan , ...","project scope statement , project documents ...","expert judgment , data analysis , decision ..."
3,create wbs,create wbs is the process of subdividing proje...,produce west_by_south,"Figure 5-10 , Figure 5-11","project management plan , project documents ...","scope baseline , project documents updates","expert judgment , decomposition"
4,validate scope,validate scope is the process of formalizing a...,corroborate range,"Figure 5-16 , Figure 5-15",,,
5,control scope,control scope is the process of monitoring the...,dominance range,"Figure 5-17 , Figure 5-18","project management plan , project documents ...","work performance information , change reques...",data analysis
6,plan schedule management,plan schedule management is the process of est...,program agenda direction,"Figure 6-3 , Figure 6-4","project charter , project management plan , ...",schedule management plan,"expert judgment , data analysis , meetings"
7,define activities,deﬁne activities is the process of identifying...,delimit activity,"Figure 6-5 , Figure 6-6","project management plan , enterprise environ...","activity list , activity attributes , miles...","expert judgment , decomposition , rolling w..."
8,sequence activities,sequence activities is the process of identify...,episode activity,"Figure 6-8 , Figure 6-7","project management plan , project documents ...","project schedule network diagrams , project ...","precedence diagramming method , dependency d..."
9,estimate activity durations,estimate activity durations is the process of ...,approximation action duration,"Figure 6-13 , Figure 6-12","project management plan , project documents ...","duration estimates , basis of estimates , p...",


In [8]:
!pip install faiss-gpu


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 85.5 MB 111 kB/s 


In [9]:
from transformers import T5Tokenizer, T5ForConditionalGeneration
import torch
tokenizer = T5Tokenizer.from_pretrained('BeIR/query-gen-msmarco-t5-large-v1')
model = T5ForConditionalGeneration.from_pretrained('BeIR/query-gen-msmarco-t5-large-v1')
#Select the device
device = 'cuda'
model.to(device)
paragraphs=data_process_name.Definition.tolist()+data_process_name.synonyms.tolist()+data_process_name.Data_property.tolist()+data_process_name.process_name.tolist()+data_process_name.inputs.tolist()+data_process_name.outputs.tolist()+data_process_name.technics_and_tools.tolist()
# Parameters for generation
batch_size = 16 #Batch size
num_queries = 5 #Number of queries to generate for every paragraph
max_length_paragraph = 512 #Max length for paragraph
max_length_query = 64   #Max length for output query
def _removeNonAscii(s): return "".join(i for i in s if ord(i) < 128)
with open('generated_queries_all.tsv', 'w') as fOut:
    for start_idx in tqdm(range(0, len(paragraphs), batch_size)):
        sub_paragraphs = paragraphs[start_idx:start_idx+batch_size]
        inputs = tokenizer.prepare_seq2seq_batch(sub_paragraphs, max_length=max_length_paragraph, truncation=True, return_tensors='pt').to(device)
        outputs = model.generate(
            **inputs,
            max_length=max_length_query,
            do_sample=True,
            top_p=0.95,
            num_return_sequences=num_queries)

        for idx, out in enumerate(outputs):
            query = tokenizer.decode(out, skip_special_tokens=True)
            query = _removeNonAscii(query)
            para = sub_paragraphs[int(idx/num_queries)]
            para = _removeNonAscii(para)
            fOut.write("{}\t{}\n".format(query.replace("\t", " ").strip(), para.replace("\t", " ").strip()))

Downloading:   0%|          | 0.00/792k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.79k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.86k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.38k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.95G [00:00<?, ?B/s]

  0%|          | 0/7 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:3672: FutureWarning: 
`prepare_seq2seq_batch` is deprecated and will be removed in version 5 of HuggingFace Transformers. Use the regular
`__call__` method to prepare your inputs and targets.

Here is a short example:

model_inputs = tokenizer(src_texts, text_target=tgt_texts, ...)

If you either need to use different keyword arguments for the source and target texts, you should do two calls like
this:

model_inputs = tokenizer(src_texts, ...)
labels = tokenizer(text_target=tgt_texts, ...)
model_inputs["labels"] = labels["input_ids"]

See the documentation of your specific tokenizer for more details on the specific arguments to the tokenizer of choice.
For a more complete example, see the implementation of `prepare_seq2seq_batch`.

  warnings.warn(formatted_warning, FutureWarning)
100%|██████████| 7/7 [00:16<00:00,  2.37s/it]


In [10]:
from sentence_transformers import SentenceTransformer, InputExample, losses, models, datasets,evaluation
from torch import nn
import os
import random

train_examples = [] 
with open('generated_queries_all.tsv') as fIn:
    for line in fIn:
        try:
            query, paragraph = line.strip().split('\t', maxsplit=1)
            train_examples.append(InputExample(texts=[query, paragraph]))
        except:
            pass
        
random.shuffle(train_examples)

# For the MultipleNegativesRankingLoss, it is important
# that the batch does not contain duplicate entries, i.e.
# no two equal queries and no two equal paragraphs.
# To ensure this, we use a special data loader
train_dataloader = datasets.NoDuplicatesDataLoader(train_examples, batch_size=8)

# Now we create a SentenceTransformer model from scratch
word_emb = models.Transformer('sentence-transformers/msmarco-distilbert-base-dot-prod-v3')
pooling = models.Pooling(word_emb.get_word_embedding_dimension())
model = SentenceTransformer(modules=[word_emb, pooling])


# MultipleNegativesRankingLoss requires input pairs (query, relevant_passage)
# and trains the model so that is is suitable for semantic search
train_loss = losses.MultipleNegativesRankingLoss(model)


#Tune the model
num_epochs = 20
warmup_steps = int(len(train_dataloader) * num_epochs * 0.1)
model.fit(train_objectives=[(train_dataloader, train_loss)], epochs=num_epochs, warmup_steps=warmup_steps, show_progress_bar=True)

os.makedirs('search', exist_ok=True)
model.save('search/search-model')

Downloading:   0%|          | 0.00/554 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/265M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/376 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

...

Iteration:   0%|          | 0/65 [00:00<?, ?it/s]

Iteration:   0%|          | 0/65 [00:00<?, ?it/s]

Iteration:   0%|          | 0/65 [00:00<?, ?it/s]

Iteration:   0%|          | 0/65 [00:00<?, ?it/s]

Iteration:   0%|          | 0/65 [00:00<?, ?it/s]

In [11]:
from sentence_transformers import SentenceTransformer, util
import gzip
import json
import os

model= SentenceTransformer('search/search-model')


In [12]:
!zip -r search_model_process_name.zip "./search"

  adding: search/ (stored 0%)
  adding: search/search-model/ (stored 0%)
  adding: search/search-model/1_Pooling/ (stored 0%)
  adding: search/search-model/1_Pooling/config.json (deflated 47%)
  adding: search/search-model/config_sentence_transformers.json (deflated 26%)
  adding: search/search-model/special_tokens_map.json (deflated 42%)
  adding: search/search-model/vocab.txt (deflated 53%)
  adding: search/search-model/tokenizer_config.json (deflated 44%)
  adding: search/search-model/pytorch_model.bin (deflated 8%)
  adding: search/search-model/sentence_bert_config.json (deflated 4%)
  adding: search/search-model/config.json (deflated 44%)
  adding: search/search-model/tokenizer.json (deflated 71%)
  adding: search/search-model/README.md (deflated 58%)
  adding: search/search-model/modules.json (deflated 53%)


In [13]:
################################################################End model process_name #####################################################

In [14]:
list_concepts=[]
for elm in data_concepts['concepts']:
  if elm not in list_concepts:
    list_concepts.append(elm)
list_concepts

[' project charter',
 ' project management plan',
 ' enterprise environmental factors',
 ' organizational process assets',
 ' expert judgment',
 ' data analysis',
 ' meetings',
 ' scope management plan',
 ' requirements management plan',
 ' project documents',
 ' business documents',
 ' agreements',
 ' data gathering',
 ' decision making',
 ' data representation',
 ' interpersonal and team skills',
 ' context diagram',
 ' prototypes',
 ' requirements documentation',
 ' requirements traceability matrix',
 ' product analysis',
 ' project scope statement',
 ' project documents updates',
 ' decomposition',
 ' scope baseline',
 ' work performance data',
 ' work performance information',
 ' change requests',
 ' project management plan updates',
 ' schedule management plan',
 ' rolling wave planning',
 ' activity list',
 ' activity attributes',
 ' milestone list',
 ' precedence diagramming method',
 ' dependency determination and integration',
 ' leads and lags',
 ' project management informa

In [15]:
list_concepts_process_name_inputs=[]
list_concepts_process_name_outputs=[]
list_concepts_process_name_technics_and_tools=[]
list_concepts_def=[]
list_concepts_syn=[]
list_concepts_data_property=[]
for elm in list_concepts:
  text_in=''
  text_ou=''
  text_tt=''
  text_def=''
  text_syn=''
  text_dt=''
  for i in data_concepts.index:
    if data_concepts.loc[i,'concepts']==elm and data_concepts.loc[i,'type']=='Has_Inputs':
      text_in+=data_concepts.loc[i,'process_name']+' , '
    if data_concepts.loc[i,'concepts']==elm and data_concepts.loc[i,'type']=='Has_Outputs':
      text_ou+=data_concepts.loc[i,'process_name']+' , '
    if data_concepts.loc[i,'concepts']==elm and data_concepts.loc[i,'type']=='Has_Technics_Tools':
      text_tt+=data_concepts.loc[i,'process_name']+' , '
    if data_concepts.loc[i,'concepts']==elm :
      text_def=data_concepts.loc[i,'Definition']
      text_syn=data_concepts.loc[i,'synonyms']
      if pd.isna(data_concepts.loc[i,'DataProperty'])==False:
        text_dt=data_concepts.loc[i,'DataProperty']
      else:
        text_dt=''


  list_concepts_process_name_inputs.append(text_in[:-2])
  list_concepts_process_name_outputs.append(text_ou[:-2])
  list_concepts_process_name_technics_and_tools.append(text_tt[:-2])
  list_concepts_def.append(text_def)
  list_concepts_syn.append(text_syn)
  list_concepts_data_property.append(text_dt)


print('inputs: ',list_concepts_process_name_inputs)
print('outputs: ',list_concepts_process_name_outputs)
print('technics & tools: ',list_concepts_process_name_technics_and_tools)
print('definition: ',list_concepts_def)
print('synonyms: ',list_concepts_syn)
print('data_property: ',list_concepts_data_property)

inputs:  [' plan scope management ,  collect requirements ,  define scope ,  plan schedule management ,  plan cost management ', ' plan scope management ,  collect requirements ,  define scope ,  create wbs ,  control scope ,  plan schedule management ,  define activities ,  sequence activities ,  estimate activity durations ,  develop schedule ,  plan cost management ,  estimate costs ,  determine budget ,  control costs ', ' plan scope management ,  collect requirements ,  define scope ,  create wbs ,  plan schedule management ,  define activities ,  sequence activities ,  estimate activity durations ,  develop schedule ,  plan cost management ,  estimate costs ,  determine budget ', ' plan scope management ,  collect requirements ,  define scope ,  create wbs ,  control scope ,  plan schedule management ,  define activities ,  sequence activities ,  estimate activity durations ,  develop schedule ,  plan cost management ,  estimate costs ,  determine budget ,  control costs ', '', '

In [16]:
dict_concepts={}
dict_concepts['concepts']=list_concepts
dict_concepts['synonyms']=list_concepts_syn
dict_concepts['Definition']=list_concepts_def
dict_concepts['process_inputs']=list_concepts_process_name_inputs
dict_concepts['process_outputs']=list_concepts_process_name_outputs
dict_concepts['process_technics_and_tools']=list_concepts_process_name_technics_and_tools
dict_concepts['Data_property']=list_concepts_data_property
data_concepts_rec=pd.DataFrame(dict_concepts)
data_concepts_rec

,concepts,synonyms,Definition,process_inputs,process_outputs,process_technics_and_tools,Data_property
0,project charter,undertaking lease,the project charter provides the preapproved...,"plan scope management , collect requirements...",,,section 4.2.3.1
1,project management plan,undertaking direction program,project management plan components include b...,"plan scope management , collect requirements...",,,section 4.2.3.1
2,enterprise environmental factors,endeavor environmental factor,the enterprise environmental factors that can ...,"plan scope management , collect requirements...",,,
3,organizational process assets,organisational procedure asset,the organizational process assets that can inﬂ...,"plan scope management , collect requirements...",,,
4,expert judgment,adept opinion,examples of expert judgment during the contr...,,,"plan scope management , collect requirements...",section 4.1.2.1
...,...,...,...,...,...,...,...
59,financing,finance,financing entails acquiring funding for projec...,,,determine budget,
60,cost baseline,toll service_line,the cost baseline is the approved version of t...,,determine budget,,section 7.2.2.6
61,project funding requirements,undertaking financial_support requirement,the project funding requirements include pro...,control costs,determine budget,,section 7.3.3.2
62,tocomplete performance index,public_presentation indicant,the tocomplete performance index tcpi is a mea...,,,control costs,


In [17]:
from transformers import T5Tokenizer, T5ForConditionalGeneration
import torch
tokenizer = T5Tokenizer.from_pretrained('BeIR/query-gen-msmarco-t5-large-v1')
model = T5ForConditionalGeneration.from_pretrained('BeIR/query-gen-msmarco-t5-large-v1')
#Select the device
device = 'cuda'
model.to(device)
paragraphs=data_concepts_rec.Definition.tolist()+data_concepts_rec.synonyms.tolist()+data_concepts_rec.Data_property.tolist()+data_concepts_rec.concepts.tolist()+data_concepts_rec.process_inputs.tolist()+data_concepts_rec.process_outputs.tolist()+data_concepts_rec.process_technics_and_tools.tolist()
# Parameters for generation
batch_size = 16 #Batch size
num_queries = 5 #Number of queries to generate for every paragraph
max_length_paragraph = 512 #Max length for paragraph
max_length_query = 64   #Max length for output query
def _removeNonAscii(s): return "".join(i for i in s if ord(i) < 128)
with open('generated_queries_all.tsv', 'w') as fOut:
    for start_idx in tqdm(range(0, len(paragraphs), batch_size)):
        sub_paragraphs = paragraphs[start_idx:start_idx+batch_size]
        inputs = tokenizer.prepare_seq2seq_batch(sub_paragraphs, max_length=max_length_paragraph, truncation=True, return_tensors='pt').to(device)
        outputs = model.generate(
            **inputs,
            max_length=max_length_query,
            do_sample=True,
            top_p=0.95,
            num_return_sequences=num_queries)

        for idx, out in enumerate(outputs):
            query = tokenizer.decode(out, skip_special_tokens=True)
            query = _removeNonAscii(query)
            para = sub_paragraphs[int(idx/num_queries)]
            para = _removeNonAscii(para)
            fOut.write("{}\t{}\n".format(query.replace("\t", " ").strip(), para.replace("\t", " ").strip()))

  0%|          | 0/28 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:3672: FutureWarning: 
`prepare_seq2seq_batch` is deprecated and will be removed in version 5 of HuggingFace Transformers. Use the regular
`__call__` method to prepare your inputs and targets.

Here is a short example:

model_inputs = tokenizer(src_texts, text_target=tgt_texts, ...)

If you either need to use different keyword arguments for the source and target texts, you should do two calls like
this:

model_inputs = tokenizer(src_texts, ...)
labels = tokenizer(text_target=tgt_texts, ...)
model_inputs["labels"] = labels["input_ids"]

See the documentation of your specific tokenizer for more details on the specific arguments to the tokenizer of choice.
For a more complete example, see the implementation of `prepare_seq2seq_batch`.

  warnings.warn(formatted_warning, FutureWarning)
100%|██████████| 28/28 [01:01<00:00,  2.21s/it]


In [19]:
from sentence_transformers import SentenceTransformer, InputExample, losses, models, datasets,evaluation
from torch import nn
import os
import random

train_examples = [] 
with open('generated_queries_all.tsv') as fIn:
    for line in fIn:
        try:
            query, paragraph = line.strip().split('\t', maxsplit=1)
            train_examples.append(InputExample(texts=[query, paragraph]))
        except:
            pass
        
random.shuffle(train_examples)

# For the MultipleNegativesRankingLoss, it is important
# that the batch does not contain duplicate entries, i.e.
# no two equal queries and no two equal paragraphs.
# To ensure this, we use a special data loader
train_dataloader = datasets.NoDuplicatesDataLoader(train_examples, batch_size=8)

# Now we create a SentenceTransformer model from scratch
word_emb = models.Transformer('sentence-transformers/msmarco-distilbert-base-dot-prod-v3')
pooling = models.Pooling(word_emb.get_word_embedding_dimension())
model = SentenceTransformer(modules=[word_emb, pooling])


# MultipleNegativesRankingLoss requires input pairs (query, relevant_passage)
# and trains the model so that is is suitable for semantic search
train_loss = losses.MultipleNegativesRankingLoss(model)


#Tune the model
num_epochs = 20
warmup_steps = int(len(train_dataloader) * num_epochs * 0.1)
model.fit(train_objectives=[(train_dataloader, train_loss)], epochs=num_epochs, warmup_steps=warmup_steps, show_progress_bar=True)

os.makedirs('search', exist_ok=True)
model.save('search/search-model')

Epoch:   0%|          | 0/20 [00:00<?, ?it/s]

Iteration:   0%|          | 0/181 [00:00<?, ?it/s]

Iteration:   0%|          | 0/181 [00:00<?, ?it/s]

Iteration:   0%|          | 0/181 [00:00<?, ?it/s]

Iteration:   0%|          | 0/181 [00:00<?, ?it/s]

...

Iteration:   0%|          | 0/181 [00:00<?, ?it/s]

Iteration:   0%|          | 0/181 [00:00<?, ?it/s]

Iteration:   0%|          | 0/181 [00:00<?, ?it/s]

Iteration:   0%|          | 0/181 [00:00<?, ?it/s]

Iteration:   0%|          | 0/181 [00:00<?, ?it/s]

In [20]:
from sentence_transformers import SentenceTransformer, util
import gzip
import json
import os

model= SentenceTransformer('search/search-model')


In [21]:
!zip -r search_model_concept.zip "./search"

  adding: search/ (stored 0%)
  adding: search/search-model/ (stored 0%)
  adding: search/search-model/1_Pooling/ (stored 0%)
  adding: search/search-model/1_Pooling/config.json (deflated 47%)
  adding: search/search-model/config_sentence_transformers.json (deflated 26%)
  adding: search/search-model/special_tokens_map.json (deflated 42%)
  adding: search/search-model/vocab.txt (deflated 53%)
  adding: search/search-model/tokenizer_config.json (deflated 44%)
  adding: search/search-model/pytorch_model.bin (deflated 8%)
  adding: search/search-model/sentence_bert_config.json (deflated 4%)
  adding: search/search-model/config.json (deflated 44%)
  adding: search/search-model/tokenizer.json (deflated 71%)
  adding: search/search-model/README.md (deflated 58%)
  adding: search/search-model/modules.json (deflated 53%)


In [22]:
################################################### end concepts model #############################################################

In [27]:
from transformers import T5Tokenizer, T5ForConditionalGeneration
import torch
tokenizer = T5Tokenizer.from_pretrained('BeIR/query-gen-msmarco-t5-large-v1')
model = T5ForConditionalGeneration.from_pretrained('BeIR/query-gen-msmarco-t5-large-v1')
#Select the device
device = 'cuda'
model.to(device)
paragraphs=data_object_property.Domaine.tolist()+data_object_property.Object_Property.tolist()+data_object_property.Range.tolist()
# Parameters for generation
batch_size = 16 #Batch size
num_queries = 5 #Number of queries to generate for every paragraph
max_length_paragraph = 512 #Max length for paragraph
max_length_query = 64   #Max length for output query
# def _removeNonAscii(s): return "".join(i for i in s if ord(i) < 128)
with open('generated_queries_all.tsv', 'w') as fOut:
    for start_idx in tqdm(range(0, len(paragraphs), batch_size)):
        sub_paragraphs = paragraphs[start_idx:start_idx+batch_size]
        inputs = tokenizer.prepare_seq2seq_batch(str(sub_paragraphs), max_length=max_length_paragraph, truncation=True, return_tensors='pt').to(device)
        outputs = model.generate(
            **inputs,
            max_length=max_length_query,
            do_sample=True,
            top_p=0.95,
            num_return_sequences=num_queries)

        for idx, out in enumerate(outputs):
            query = tokenizer.decode(out, skip_special_tokens=True)
            query = _removeNonAscii(query)
            para = sub_paragraphs[int(idx/num_queries)]
            para = str(para)
            fOut.write("{}\t{}\n".format(query.replace("\t", " ").strip(), para.replace("\t", " ").strip()))

100%|██████████| 125/125 [01:02<00:00,  2.00it/s]


In [28]:
from sentence_transformers import SentenceTransformer, InputExample, losses, models, datasets,evaluation
from torch import nn
import os
import random

train_examples = [] 
with open('generated_queries_all.tsv') as fIn:
    for line in fIn:
        try:
            query, paragraph = line.strip().split('\t', maxsplit=1)
            train_examples.append(InputExample(texts=[query, paragraph]))
        except:
            pass
        
random.shuffle(train_examples)

# For the MultipleNegativesRankingLoss, it is important
# that the batch does not contain duplicate entries, i.e.
# no two equal queries and no two equal paragraphs.
# To ensure this, we use a special data loader
train_dataloader = datasets.NoDuplicatesDataLoader(train_examples, batch_size=8)

# Now we create a SentenceTransformer model from scratch
word_emb = models.Transformer('sentence-transformers/msmarco-distilbert-base-dot-prod-v3')
pooling = models.Pooling(word_emb.get_word_embedding_dimension())
model = SentenceTransformer(modules=[word_emb, pooling])


# MultipleNegativesRankingLoss requires input pairs (query, relevant_passage)
# and trains the model so that is is suitable for semantic search
train_loss = losses.MultipleNegativesRankingLoss(model)


#Tune the model
num_epochs = 20
warmup_steps = int(len(train_dataloader) * num_epochs * 0.1)
model.fit(train_objectives=[(train_dataloader, train_loss)], epochs=num_epochs, warmup_steps=warmup_steps, show_progress_bar=True)

os.makedirs('search', exist_ok=True)
model.save('search/search-model')

Epoch:   0%|          | 0/20 [00:00<?, ?it/s]

Iteration:   0%|          | 0/78 [00:00<?, ?it/s]

Iteration:   0%|          | 0/78 [00:00<?, ?it/s]

Iteration:   0%|          | 0/78 [00:00<?, ?it/s]

Iteration:   0%|          | 0/78 [00:00<?, ?it/s]

...

Iteration:   0%|          | 0/78 [00:00<?, ?it/s]

Iteration:   0%|          | 0/78 [00:00<?, ?it/s]

Iteration:   0%|          | 0/78 [00:00<?, ?it/s]

Iteration:   0%|          | 0/78 [00:00<?, ?it/s]

Iteration:   0%|          | 0/78 [00:00<?, ?it/s]

In [29]:
from sentence_transformers import SentenceTransformer, util
import gzip
import json
import os

model= SentenceTransformer('search/search-model')


In [30]:
!zip -r search_model_object_property.zip "./search"

  adding: search/ (stored 0%)
  adding: search/search-model/ (stored 0%)
  adding: search/search-model/1_Pooling/ (stored 0%)
  adding: search/search-model/1_Pooling/config.json (deflated 47%)
  adding: search/search-model/config_sentence_transformers.json (deflated 26%)
  adding: search/search-model/special_tokens_map.json (deflated 42%)
  adding: search/search-model/vocab.txt (deflated 53%)
  adding: search/search-model/tokenizer_config.json (deflated 44%)
  adding: search/search-model/pytorch_model.bin (deflated 8%)
  adding: search/search-model/sentence_bert_config.json (deflated 4%)
  adding: search/search-model/config.json (deflated 44%)
  adding: search/search-model/tokenizer.json (deflated 71%)
  adding: search/search-model/README.md (deflated 58%)
  adding: search/search-model/modules.json (deflated 53%)


In [ ]:
############################################## end object property model #####################################################################

In [ ]:
##############################################test process name model #############################################

In [34]:
import zipfile
with zipfile.ZipFile('search_model_process_name.zip', 'r') as zip_ref:
    zip_ref.extractall('test/')

In [35]:
model= SentenceTransformer('test/search/search-model')


In [36]:
import faiss
encoded_data1 = model.encode(data_process_name.Definition.tolist())
encoded_data1 = np.asarray(encoded_data1.astype('float32'))
encoded_data2 = model.encode(data_process_name.process_name.tolist())
encoded_data2 = np.asarray(encoded_data2.astype('float32'))
encoded_data3 = model.encode(data_process_name.synonyms.tolist())
encoded_data3 = np.asarray(encoded_data3.astype('float32'))
encoded_data4 = model.encode(data_process_name.inputs.tolist())
encoded_data4 = np.asarray(encoded_data4.astype('float32'))
encoded_data5 = model.encode(data_process_name.outputs.tolist())
encoded_data5 = np.asarray(encoded_data5.astype('float32'))
encoded_data6 = model.encode(data_process_name.technics_and_tools.tolist())
encoded_data6 = np.asarray(encoded_data6.astype('float32'))
encoded_data7 = model.encode(data_process_name.Data_property.tolist())
encoded_data7 = np.asarray(encoded_data7.astype('float32'))
index = faiss.IndexIDMap(faiss.IndexFlatIP(768))
index.add_with_ids(encoded_data1, np.array(range(0, len(data_process_name))))
index.add_with_ids(encoded_data2, np.array(range(0, len(data_process_name))))
index.add_with_ids(encoded_data3, np.array(range(0, len(data_process_name))))
index.add_with_ids(encoded_data4, np.array(range(0, len(data_process_name))))
index.add_with_ids(encoded_data5, np.array(range(0, len(data_process_name))))
index.add_with_ids(encoded_data6, np.array(range(0, len(data_process_name))))
index.add_with_ids(encoded_data7, np.array(range(0, len(data_process_name))))
faiss.write_index(index, 'process_name.index')

In [37]:
def fetch_process_name_info(dataframe_idx):
    info = data_process_name.iloc[dataframe_idx]
    meta_dict = {}
    meta_dict['process_name'] = info['process_name']
    meta_dict['synonyms'] = info['synonyms']
    meta_dict['Definition']=info['Definition']
    meta_dict['Data_property']=info['Data_property']
    meta_dict['inputs']=info['inputs']
    meta_dict['outputs']=info['outputs']
    meta_dict['technics_and_tools']=info['technics_and_tools']
    return meta_dict
    
def search_process_name(query, top_k, index, model):
    t=time.time()
    query_vector = model.encode([query])
    top_k = index.search(query_vector, top_k)
    print('>>>> Results in Total Time: {}'.format(time.time()-t))
    top_k_ids = top_k[1].tolist()[0]
    top_k_ids = list(np.unique(top_k_ids))
    results =  [fetch_process_name_info(idx) for idx in top_k_ids]
    return results

In [39]:
from pprint import pprint

query="plan schedule what is  "
results=search_process_name(query, top_k=5, index=index, model=model)

print("\n")
for result in results:
    print('\t',result)


>>>> Results in Total Time: 0.020969867706298828


	 {'process_name': ' plan schedule management', 'synonyms': 'program agenda direction ', 'Definition': 'plan schedule management is the process of establishing the policies procedures and documentation for  planning developing managing executing and controlling the project schedule the key beneﬁt of this process is  that it provides guidance and direction on how the project schedule will be managed throughout the project this  process is performed once or at predeﬁned points in the project', 'Data_property': 'Figure 6-3 , Figure 6-4 ', 'inputs': ' project charter ,  project management plan ,  enterprise environmental factors ,  organizational process assets ', 'outputs': ' schedule management plan ', 'technics_and_tools': ' expert judgment ,  data analysis ,  meetings '}
	 {'process_name': ' sequence activities', 'synonyms': 'episode activity ', 'Definition': 'sequence activities is the process of identifying and documenting relationsh

In [ ]:
############################################## fin test process_name##########################################################

In [ ]:
############################################ test concept ######################################################

In [44]:
import zipfile
with zipfile.ZipFile('search_model_concept.zip', 'r') as zip_ref:
    zip_ref.extractall('test/')

In [45]:
model= SentenceTransformer('test/search/search-model')


In [46]:
import faiss
encoded_data1 = model.encode(data_concepts_rec.Definition.tolist())
encoded_data1 = np.asarray(encoded_data1.astype('float32'))
encoded_data2 = model.encode(data_concepts_rec.concepts.tolist())
encoded_data2 = np.asarray(encoded_data2.astype('float32'))
encoded_data3 = model.encode(data_concepts_rec.synonyms.tolist())
encoded_data3 = np.asarray(encoded_data3.astype('float32'))
encoded_data4 = model.encode(data_concepts_rec.process_inputs.tolist())
encoded_data4 = np.asarray(encoded_data4.astype('float32'))
encoded_data5 = model.encode(data_concepts_rec.process_outputs.tolist())
encoded_data5 = np.asarray(encoded_data5.astype('float32'))
encoded_data6 = model.encode(data_concepts_rec.process_technics_and_tools.tolist())
encoded_data6 = np.asarray(encoded_data6.astype('float32'))
encoded_data7 = model.encode(data_concepts_rec.Data_property.tolist())
encoded_data7 = np.asarray(encoded_data7.astype('float32'))
index = faiss.IndexIDMap(faiss.IndexFlatIP(768))
index.add_with_ids(encoded_data1, np.array(range(0, len(data_concepts_rec))))
index.add_with_ids(encoded_data2, np.array(range(0, len(data_concepts_rec))))
index.add_with_ids(encoded_data3, np.array(range(0, len(data_concepts_rec))))
index.add_with_ids(encoded_data4, np.array(range(0, len(data_concepts_rec))))
index.add_with_ids(encoded_data5, np.array(range(0, len(data_concepts_rec))))
index.add_with_ids(encoded_data6, np.array(range(0, len(data_concepts_rec))))
index.add_with_ids(encoded_data7, np.array(range(0, len(data_concepts_rec))))

faiss.write_index(index, 'concepts.index')

In [47]:
def fetch_concepts_info(dataframe_idx):
    info = data_concepts_rec.iloc[dataframe_idx]
    meta_dict = {}
    meta_dict['concepts'] = info['concepts']
    meta_dict['synonyms'] = info['synonyms']
    meta_dict['Definition']=info['Definition']
    meta_dict['process_inputs']=info['process_inputs']
    meta_dict['process_outputs']=info['process_outputs']
    meta_dict['process_technics_and_tools']=info['process_technics_and_tools']
    meta_dict['Data_property']=info['Data_property']
    return meta_dict
    
def search_concepts(query, top_k, index, model):
    t=time.time()
    query_vector = model.encode([query])
    top_k = index.search(query_vector, top_k)
    print('>>>> Results in Total Time: {}'.format(time.time()-t))
    top_k_ids = top_k[1].tolist()[0]
    top_k_ids = list(np.unique(top_k_ids))
    results =  [fetch_concepts_info(idx) for idx in top_k_ids]
    return results

In [48]:
from pprint import pprint

query="enterprise environmental factors		 is "
results=search_concepts(query, top_k=5, index=index, model=model)

print("\n")
for result in results:
    print('\t',result)


>>>> Results in Total Time: 0.013642549514770508


	 {'concepts': ' enterprise environmental factors', 'synonyms': 'endeavor environmental factor ', 'Definition': 'the enterprise environmental factors that can inﬂuence the estimate costs process include but are not limited  to exchange rates for largescale projects that extend multiple years with multiple currencies the ﬂuctuations of  currencies need to be understood and built into the determine budget process', 'process_inputs': ' plan scope management ,  collect requirements ,  define scope ,  create wbs ,  plan schedule management ,  define activities ,  sequence activities ,  estimate activity durations ,  develop schedule ,  plan cost management ,  estimate costs ,  determine budget ', 'process_outputs': '', 'process_technics_and_tools': '', 'Data_property': ''}
	 {'concepts': ' activity attributes', 'synonyms': 'action attribute ', 'Definition': 'activity attributes extend the description of the activity by identifying multiple 

In [ ]:
#########################################################fin test concept######################################################################

In [ ]:
##########################################################test onject property ################################################################

In [49]:
import zipfile
with zipfile.ZipFile('search_model_object_property.zip', 'r') as zip_ref:
    zip_ref.extractall('test/')

In [50]:
model= SentenceTransformer('test/search/search-model')


In [51]:
import faiss
encoded_data1 = model.encode(data_object_property.Domaine.tolist())
encoded_data1 = np.asarray(encoded_data1.astype('float32'))
encoded_data2 = model.encode(data_object_property.Object_Property.tolist())
encoded_data2 = np.asarray(encoded_data2.astype('float32'))
encoded_data3 = model.encode(data_object_property.Range.tolist())
encoded_data3 = np.asarray(encoded_data3.astype('float32'))

index = faiss.IndexIDMap(faiss.IndexFlatIP(768))
index.add_with_ids(encoded_data1, np.array(range(0, len(data_object_property))))
index.add_with_ids(encoded_data2, np.array(range(0, len(data_object_property))))
index.add_with_ids(encoded_data3, np.array(range(0, len(data_object_property))))


faiss.write_index(index, 'object.index')

In [52]:
def fetch_object_info(dataframe_idx):
    info = data_object_property.iloc[dataframe_idx]
    meta_dict = {}
    meta_dict['Domaine'] = info['Domaine']
    meta_dict['Object_Property'] = info['Object_Property']
    meta_dict['Range']=info['Range']

    return meta_dict
    
def search_object(query, top_k, index, model):
    t=time.time()
    query_vector = model.encode([query])
    top_k = index.search(query_vector, top_k)
    print('>>>> Results in Total Time: {}'.format(time.time()-t))
    top_k_ids = top_k[1].tolist()[0]
    top_k_ids = list(np.unique(top_k_ids))
    results =  [fetch_object_info(idx) for idx in top_k_ids]
    return results

In [54]:
from pprint import pprint

query="risk element "
results=search_object(query, top_k=5, index=index, model=model)

print("\n")
for result in results:
    print('\t',result)


>>>> Results in Total Time: 0.02163100242614746


	 {'Domaine': 'approach ', 'Object_Property': 'establish', 'Range': 'criterion risk level '}
	 {'Domaine': 'speciﬁcity information project ', 'Object_Property': 'knowexist', 'Range': 'risk assumption '}
	 {'Domaine': 'risk source uncertainty ', 'Object_Property': 'use', 'Range': 'calculate '}
	 {'Domaine': 'risk issue scenario ', 'Object_Property': 'use', 'Range': 'probability distribution '}
	 {'Domaine': 'activity assumption ', 'Object_Property': 'constraint', 'Range': 'risk issue scenario '}


In [ ]:
###############################################end object test ###########################################################################

In [ ]:
##############################################auto completion model using Ngram Model ###############################################

In [61]:
sentences=[]
for i in data_concepts_rec.index:
  for col in data_concepts_rec.columns:
    sentences.append(str(data_concepts_rec.loc[i,col]))
for i in data_process_name.index:
  for col in data_process_name.columns:
    sentences.append(str(data_process_name.loc[i,col]))
for i in data_object_property.index:
  for col in data_object_property.columns:
    sentences.append(str(data_object_property.loc[i,col]))

In [63]:
import nltk
nltk.download('punkt')
def preprocess_pipeline(sentences) -> 'list':

 
    
    # Remove leading and trailing spaces
    sentences = [s.strip() for s in sentences]
    
    # Drop Empty Sentences
    sentences = [s for s in sentences if len(s) > 0]
    
    # Empty List to hold Tokenized Sentences
    tokenized = []
    
    # Iterate through sentences
    for sentence in sentences:
        
        # Convert to lowercase
        sentence = sentence.lower()
        
        # Convert to a list of words
        token = nltk.word_tokenize(sentence)
        
        # Append to list
        tokenized.append(token)
        
    return tokenized


## Pass our data to this function    
tokenized_sentences = preprocess_pipeline(sentences)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
tokenized_sentences

In [65]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(tokenized_sentences, test_size=0.2, random_state=42)
train, val = train_test_split(train, test_size=0.25, random_state=42)



In [66]:
def count_the_words(sentences) -> 'dict':
    
  # Creating a Dictionary of counts
  word_counts = {}

  # Iterating over sentences
  for sentence in sentences:
    
    # Iterating over Tokens
    for token in sentence:
    
      # Add count for new word
      if token not in word_counts.keys():
        word_counts[token] = 1
        
      # Increase count by one
      else:
        word_counts[token] += 1
        
  return word_counts



In [67]:
def handling_oov(tokenized_sentences, count_threshold) -> 'list':

  # Empty list for closed vocabulary
  closed_vocabulary = []

  # Obtain frequency dictionary using previously defined function
  words_count = count_the_words(tokenized_sentences)
    
  # Iterate over words and counts 
  for word, count in words_count.items():
    
    # Append if it's more(or equal) to the threshold 
    if count >= count_threshold :
      closed_vocabulary.append(word)

  return closed_vocabulary

In [68]:
def unk_tokenize(tokenized_sentences, vocabulary, unknown_token = "<unk>") -> 'list':

  # Convert Vocabulary into a set
  vocabulary = set(vocabulary)

  # Create empty list for sentences
  new_tokenized_sentences = []
  
  # Iterate over sentences
  for sentence in tokenized_sentences:

    # Iterate over sentence and add <unk> 
    # if the token is absent from the vocabulary
    new_sentence = []
    for token in sentence:
      if token in vocabulary:
        new_sentence.append(token)
      else:
        new_sentence.append(unknown_token)
    
    # Append sentece to the new list
    new_tokenized_sentences.append(new_sentence)

  return new_tokenized_sentences

In [69]:
def cleansing(train_data, test_data, count_threshold):
    
  # Get closed Vocabulary
  vocabulary = handling_oov(train_data, count_threshold)
    
  # Updated Training Dataset
  new_train_data = unk_tokenize(train_data, vocabulary)
    
  # Updated Test Dataset
  new_test_data = unk_tokenize(test_data, vocabulary)

  return new_train_data, new_test_data, vocabulary

In [70]:
min_freq = 6
final_train, final_test, vocabulary = cleansing(train, test, min_freq)



In [71]:
def count_n_grams(data, n, start_token = "<s>", end_token = "<e>") -> 'dict':

  # Empty dict for n-grams
  n_grams = {}
 
  # Iterate over all sentences in the dataset
  for sentence in data:
        
    # Append n start tokens and a single end token to the sentence
    sentence = [start_token]*n + sentence + [end_token]
    
    # Convert the sentence into a tuple
    sentence = tuple(sentence)

    # Temp var to store length from start of n-gram to end
    m = len(sentence) if n==1 else len(sentence)-1
    
    # Iterate over this length
    for i in range(m):
        
      # Get the n-gram
      n_gram = sentence[i:i+n]
    
      # Add the count of n-gram as value to our dictionary
      # IF n-gram is already present
      if n_gram in n_grams.keys():
        n_grams[n_gram] += 1
      # Add n-gram count
      else:
        n_grams[n_gram] = 1
        
  return n_grams

In [72]:
def prob_for_single_word(word, previous_n_gram, n_gram_counts, nplus1_gram_counts, vocabulary_size, k = 1.0) -> 'float':

  # Convert the previous_n_gram into a tuple 
  previous_n_gram = tuple(previous_n_gram)
    
  # Calculating the count, if exists from our freq dictionary otherwise zero
  previous_n_gram_count = n_gram_counts[previous_n_gram] if previous_n_gram in n_gram_counts else 0
  
  # The Denominator
  denom = previous_n_gram_count + k * vocabulary_size

  # previous n-gram plus the current word as a tuple
  nplus1_gram = previous_n_gram + (word,)

  # Calculating the nplus1 count, if exists from our freq dictionary otherwise zero 
  nplus1_gram_count = nplus1_gram_counts[nplus1_gram] if nplus1_gram in nplus1_gram_counts else 0

  # Numerator
  num = nplus1_gram_count + k

  # Final Fraction
  prob = num / denom
  return prob

In [73]:
def probs(previous_n_gram, n_gram_counts, nplus1_gram_counts, vocabulary, k=1.0) -> 'dict':

  # Convert to Tuple
  previous_n_gram = tuple(previous_n_gram)

  # Add end and unknown tokens to the vocabulary
  vocabulary = vocabulary + ["<e>", "<unk>"]

  # Calculate the size of the vocabulary
  vocabulary_size = len(vocabulary)

  # Empty dict for probabilites
  probabilities = {}

  # Iterate over words 
  for word in vocabulary:
    
    # Calculate probability
    probability = prob_for_single_word(word, previous_n_gram, 
                                           n_gram_counts, nplus1_gram_counts, 
                                           vocabulary_size, k=k)
    # Create mapping: word -> probability
    probabilities[word] = probability

  return probabilities

In [74]:
def auto_complete(previous_tokens, n_gram_counts, nplus1_gram_counts, vocabulary, k=1.0, start_with=None):

    
    # length of previous words
    n = len(list(n_gram_counts.keys())[0]) 
    
    # most recent 'n' words
    previous_n_gram = previous_tokens[-n:]
    
    # Calculate probabilty for all words
    probabilities = probs(previous_n_gram,n_gram_counts, nplus1_gram_counts,vocabulary, k=k)

    # Intialize the suggestion and max probability
    suggestion = None
    max_prob = 0

    # Iterate over all words and probabilites, returning the max.
    # We also add a check if the start_with parameter is provided
    for word, prob in probabilities.items():
        
        if start_with != None: 
            
            if not word.startswith(start_with):
                continue 

        if prob > max_prob: 

            suggestion = word
            max_prob = prob

    return suggestion, max_prob

In [75]:
def get_suggestions(previous_tokens, n_gram_counts_list, vocabulary, k=1.0, start_with=None):

    # See how many models we have
    count = len(n_gram_counts_list)
    
    # Empty list for suggestions
    suggestions = []
    
    # IMP: Earlier "-1"
    
    # Loop over counts
    for i in range(count-1):
        
        # get n and nplus1 counts
        n_gram_counts = n_gram_counts_list[i]
        nplus1_gram_counts = n_gram_counts_list[i+1]
        
        # get suggestions 
        suggestion = auto_complete(previous_tokens, n_gram_counts,
                                    nplus1_gram_counts, vocabulary,
                                    k=k, start_with=start_with)
        # Append to list
        suggestions.append(suggestion)
        
    return suggestions

In [76]:
n_gram_counts_list = []
for n in range(1, 6):
    n_model_counts = count_n_grams(final_train, n)
    n_gram_counts_list.append(n_model_counts)

In [83]:
phrase="meetings "
previous_tokens = nltk.word_tokenize(phrase)

suggestion = get_suggestions(previous_tokens, n_gram_counts_list, vocabulary, k=1.0)

display(suggestion)


[('project', 0.0029154518950437317),
 ('project', 0.0029154518950437317),
 ('project', 0.0029154518950437317),
 ('project', 0.0029154518950437317)]

In [78]:
print("unigram count:" , len(n_gram_counts_list[0]))
print("bigram count:", len(n_gram_counts_list[1]))
print("trigram count:", len(n_gram_counts_list[2]))
print("quadgram count:", len(n_gram_counts_list[3]))
print("quintgram count:", len(n_gram_counts_list[4]))

unigram count: 344
bigram count: 3361
trigram count: 6755
quadgram count: 8994
quintgram count: 10318
